# Problématique: identifier les zones d'accident (clustering avec DBSCAN) des 2 roues dans le département 75: PARIS

In [1]:
#import librairies
import pandas as pd
import numpy as np
from sklearn import metrics

In [2]:
#dataset: lieux-2019, usagers-2019, vehicules-2019, caracteristiques-2019
lieux_csv = "../data/raw/lieux-2019.csv"
data_lieux = pd.read_csv(lieux_csv,sep=';')
data_lieux.head()

,Num_Acc,catr,voie,v1,v2,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
0,201900000001,1,3,0.0,NaN,3,10,0,1,6,900,2,NaN,NaN,1,2,1,70
1,201900000002,1,1,0.0,NaN,1,2,0,4,3,845,2,NaN,NaN,1,0,1,70
2,201900000003,1,86,0.0,NaN,3,8,0,1,10,500,3,NaN,NaN,1,0,1,90
3,201900000004,1,4,0.0,NaN,3,5,0,1,2,299,1,NaN,NaN,1,0,1,90
4,201900000005,1,86,0.0,INT,1,3,0,1,41,0,3,NaN,NaN,1,2,1,90


In [3]:
data_lieux.shape

(58840, 18)

In [4]:
usagers_csv = "../data/raw/usagers-2019.csv"
data_usagers = pd.read_csv(usagers_csv,sep=';')
data_usagers.head()

,Num_Acc,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp
0,201900000001,138 306 524,B01,2,2,4,2,2002,0,1,0,-1,-1,-1,-1
1,201900000001,138 306 524,B01,1,1,4,2,1993,5,1,0,-1,-1,-1,-1
2,201900000001,138 306 525,A01,1,1,1,1,1959,0,1,0,-1,-1,-1,-1
3,201900000002,138 306 523,A01,1,1,4,2,1994,0,1,0,-1,-1,-1,-1
4,201900000003,138 306 520,A01,1,1,1,1,1996,0,1,0,-1,-1,0,-1


In [5]:
data_usagers.shape

(132977, 15)

Catégorie du véhicule : 00 – Indéterminable 01 – Bicyclette 02 – Cyclomoteur <50cm3 03 – Voiturette (Quadricycle à moteur carrossé) (anciennement "voiturette ou tricycle à moteur") 04 – Référence inutilisée depuis 2006 (scooter immatriculé) 05 – Référence inutilisée depuis 2006 (motocyclette) 06 – Référence inutilisée depuis 2006 (side-car) 07 – VL seul 08 – Référence inutilisée depuis 2006 (VL + caravane) 09 – Référence inutilisée depuis 2006 (VL + remorque) 10 – VU seul 1,5T <= PTAC <= 3,5T avec ou sans remorque (anciennement VU seul 1,5T <= PTAC <= 3,5T) 11 – Référence inutilisée depuis 2006 (VU (10) + caravane) 12 – Référence inutilisée depuis 2006 (VU (10) + remorque) 13 – PL seul 3,5T <PTCA <= 7,5T 14 – PL seul > 7,5T 15 – PL > 3,5T + remorque 16 – Tracteur routier seul 17 – Tracteur routier + semi-remorque 18 – Référence inutilisée depuis 2006 (transport en commun) 19 – Référence inutilisée depuis 2006 (tramway) 20 – Engin spécial 21 – Tracteur agricole 30 – Scooter < 50 cm3 31 – Motocyclette > 50 cm3 et <= 125 cm3 32 – Scooter > 50 cm3 et <= 125 cm3 33 – Motocyclette > 125 cm3 34 – Scooter > 125 cm3 35 – Quad léger <= 50 cm3 (Quadricycle à moteur non carrossé) 36 – Quad lourd > 50 cm3 (Quadricycle à moteur non carrossé) 37 – Autobus 38 – Autocar 39 – Train 40 – Tramway 41 – 3RM <= 50 cm3 42 – 3RM > 50 cm3 <= 125 cm3 43 – 3RM > 125 cm3 50 – EDP à moteur 60 – EDP sans moteur 80 – VAE 99 – Autre véhicule

In [6]:
#dat_vehicules: informations sur les types de véhicules colonne 'catv'
vehicules_csv = "../data/raw/vehicules-2019.csv"
data_vehicules = pd.read_csv(vehicules_csv,sep=';')
data_vehicules.head()

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,occutc
0,201900000001,138 306 524,B01,2,7,0,2,5,23,1,NaN
1,201900000001,138 306 525,A01,2,17,1,0,3,11,1,NaN
2,201900000002,138 306 523,A01,1,7,4,0,1,0,1,NaN
3,201900000003,138 306 520,A01,1,7,0,2,1,2,1,NaN
4,201900000003,138 306 521,B01,1,7,1,0,4,2,1,NaN


In [7]:
data_vehicules.shape

(100710, 11)

In [8]:
#data_data_caracteristiques: information localisation GPS de l'accident colonnes 'lat' et 'long'
carac_csv = "../data/raw/caracteristiques-2019.csv"
data_caracteristiques = pd.read_csv(carac_csv,sep=';')
data_caracteristiques.head()

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,201900000001,30,11,2019,01:30,4,93,93053,1,1,1,2,AUTOROUTE A3,"48,8962100","2,4701200"
1,201900000002,30,11,2019,02:50,3,93,93066,1,1,1,6,AUTOROUTE A1,"48,9307000","2,3688000"
2,201900000003,28,11,2019,15:15,1,92,92036,1,1,1,4,AUTOROUTE A86,"48,9358718","2,3191744"
3,201900000004,30,11,2019,20:20,5,94,94069,1,1,1,4,A4,"48,8173295","2,4281502"
4,201900000005,30,11,2019,04:00,3,94,94028,1,1,1,2,A86 INT,"48,7763620","2,4332540"


In [9]:
data_caracteristiques.shape

(58840, 15)

grav
Gravité de blessure de l'usager, les usagers accidentés sont classés en trois catégories de
victimes plus les indemnes :
1 – Indemne
2 – Tué
3 – Blessé hospitalisé
4 – Blessé léger

In [10]:
#fusion des données via 'Num_Acc' 
result = pd.merge(data_vehicules, data_caracteristiques, on = 'Num_Acc')

In [11]:
result.columns

Index(['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm',
       'choc', 'manv', 'motor', 'occutc', 'jour', 'mois', 'an', 'hrmn', 'lum',
       'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long'],
      dtype='object')

In [12]:
#result2 = pd.merge(result, data_usagers, on = 'Num_Acc')

In [13]:
#result2

In [14]:
result.columns

Index(['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm',
       'choc', 'manv', 'motor', 'occutc', 'jour', 'mois', 'an', 'hrmn', 'lum',
       'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long'],
      dtype='object')

In [15]:
result.head()

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,...,lum,dep,com,agg,int,atm,col,adr,lat,long
0,201900000001,138 306 524,B01,2,7,0,2,5,23,1,...,4,93,93053,1,1,1,2,AUTOROUTE A3,"48,8962100","2,4701200"
1,201900000001,138 306 525,A01,2,17,1,0,3,11,1,...,4,93,93053,1,1,1,2,AUTOROUTE A3,"48,8962100","2,4701200"
2,201900000002,138 306 523,A01,1,7,4,0,1,0,1,...,3,93,93066,1,1,1,6,AUTOROUTE A1,"48,9307000","2,3688000"
3,201900000003,138 306 520,A01,1,7,0,2,1,2,1,...,1,92,92036,1,1,1,4,AUTOROUTE A86,"48,9358718","2,3191744"
4,201900000003,138 306 521,B01,1,7,1,0,4,2,1,...,1,92,92036,1,1,1,4,AUTOROUTE A86,"48,9358718","2,3191744"


In [16]:
result.dep.value_counts()

75     9619
93     5660
13     5512
94     4853
69     4324
       ... 
90       80
978      67
977      24
986      14
975       5
Name: dep, Length: 107, dtype: int64

In [17]:
result_75 = result[result['dep']=='75']

In [18]:
result_75.head()

,Num_Acc,id_vehicule,num_veh,senc,catv,obs,obsm,choc,manv,motor,...,lum,dep,com,agg,int,atm,col,adr,lat,long
786,201900000473,138 305 660,A01,2,7,0,2,5,12,1,...,5,75,75117,2,4,1,3,BD PERIPHEDIQUE EXTERIEUR,"48,8830744","2,2837066"
787,201900000473,138 305 661,B01,2,33,0,2,3,18,1,...,5,75,75117,2,4,1,3,BD PERIPHEDIQUE EXTERIEUR,"48,8830744","2,2837066"
788,201900000474,138 305 658,B01,1,7,12,2,0,14,1,...,5,75,75120,2,2,1,3,COURS DE VINCENNES,"48,8470484","2,4110571"
789,201900000474,138 305 659,A01,1,33,0,2,3,1,1,...,5,75,75120,2,2,1,3,COURS DE VINCENNES,"48,8470484","2,4110571"
790,201900000475,138 305 656,A01,2,33,0,2,6,1,1,...,1,75,75120,2,2,1,6,RUE DE GUEBRIANT,"48,8729280","2,4086480"


In [19]:
#selecion des colones utiles
result_75 = result_75.loc[:,["lat","long","catv"]]
result_75.head()

,lat,long,catv
786,"48,8830744","2,2837066",7
787,"48,8830744","2,2837066",33
788,"48,8470484","2,4110571",7
789,"48,8470484","2,4110571",33
790,"48,8729280","2,4086480",33


In [20]:
#selectionner les 2 roues dans la colone 'catv'
values=[1,2,30,31,32,33,34,41,42,43,50,60,80]
filtered_result_75 = result_75[result_75.catv.isin(values)]
filtered_result_75.head()

,lat,long,catv
787,"48,8830744","2,2837066",33
789,"48,8470484","2,4110571",33
790,"48,8729280","2,4086480",33
792,"48,8565645","2,3944381",33
793,"48,8919220","2,3983400",32


In [21]:
filtered_result_75.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4480 entries, 787 to 100619
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lat     4480 non-null   object
 1   long    4480 non-null   object
 2   catv    4480 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 140.0+ KB


In [22]:
#traitement sur colones lat et long : remplacement ',' par '.'
filtered_result_75.lat = filtered_result_75.lat.str.replace(",",".",regex=True)
filtered_result_75.long = filtered_result_75.long.str.replace(",",".",regex=True)

/Users/mac/Desktop/Jedha_Project/env/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [23]:
filtered_result_75.head()

,lat,long,catv
787,48.8830744,2.2837066,33
789,48.8470484,2.4110571,33
790,48.8729280,2.4086480,33
792,48.8565645,2.3944381,33
793,48.8919220,2.3983400,32


In [24]:
#changer type des colones 'lat' et 'long' en numeric
filtered_result_75['lat'] = pd.to_numeric(filtered_result_75['lat'])
filtered_result_75['long'] = pd.to_numeric(filtered_result_75['long'])

/var/folders/04/nhkbrdrd3453482rr7w_x_mc0000gn/T/ipykernel_29633/3368113659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_result_75['lat'] = pd.to_numeric(filtered_result_75['lat'])
/var/folders/04/nhkbrdrd3453482rr7w_x_mc0000gn/T/ipykernel_29633/3368113659.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_result_75['long'] = pd.to_numeric(filtered_result_75['long'])


In [25]:
filtered_result_75.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4480 entries, 787 to 100619
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lat     4480 non-null   float64
 1   long    4480 non-null   float64
 2   catv    4480 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 140.0 KB


In [26]:
#affichier les points GPS sur une carte
import plotly.express as px
fig = px.scatter_mapbox(
        filtered_result_75, 
        lat="lat", 
        lon="long",
        color="catv",
        mapbox_style="carto-positron"
)

fig.show(renderer="iframe")

filtered_result_75 = pd.get_dummies(filtered_result_75)
filtered_result_75.head()

In [27]:
#identification des clusters avec DBCSAN
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.005, min_samples=6, metric="manhattan")

db.fit(filtered_result_75)

DBSCAN(eps=0.005, metric='manhattan', min_samples=6)

In [28]:
len(np.unique(db.labels_))

122

In [29]:
filtered_result_75_cluster = filtered_result_75
filtered_result_75_cluster["cluster"] = db.labels_

/var/folders/04/nhkbrdrd3453482rr7w_x_mc0000gn/T/ipykernel_29633/444248082.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
filtered_result_75_cluster["cluster"].value_counts()

-1      2379
 11      275
 5       126
 0        59
 1        59
        ... 
 110       5
 112       5
 64        4
 115       4
 114       3
Name: cluster, Length: 122, dtype: int64

In [31]:
filtered_result_75_cluster.cluster.unique()

array([  0,   1,  -1,  72,   2,  73,   3,   4,  11,   5,   6,  53,  67,
         7,   8,   9,  10,  12,  13, 106,  47,  14,  89,  91,  15,  16,
        17,  81,  18,  19,  63,  20,  21,  22,  23,  24,  25,  26,  27,
        28, 105, 107,  29,  30,  31,  32,  33,  34, 104,  35,  36,  37,
        38,  84,  39,  40,  41,  58,  42,  43,  44,  45,  46,  92, 113,
        48, 109,  49,  50,  51,  52,  78,  77,  54,  55,  56,  57,  68,
        66,  59,  60, 115,  61,  62,  64,  65, 114,  69,  70,  71, 112,
        90,  74,  75, 111,  76,  95,  79,  80, 103, 119,  82,  83,  85,
        86,  99,  87,  88, 110,  94, 117,  93, 118, 102,  96, 101,  97,
        98, 100, 120, 116, 108])

In [32]:
filtered_result_75_cluster.cluster.value_counts()

-1      2379
 11      275
 5       126
 0        59
 1        59
        ... 
 110       5
 112       5
 64        4
 115       4
 114       3
Name: cluster, Length: 122, dtype: int64

In [33]:
len(filtered_result_75_cluster[filtered_result_75_cluster['cluster']==-1])

2379

In [34]:
db.labels_

array([ 0,  1,  1, ..., 17, -1, 24])

In [35]:
type(db.labels_)

numpy.ndarray

In [36]:
metrics.silhouette_score(filtered_result_75_cluster,filtered_result_75_cluster['cluster'])

0.0817532432371763

In [37]:
fig = px.scatter_mapbox(
        filtered_result_75_cluster[filtered_result_75_cluster.cluster !=-1], 
        lat="lat", 
        lon="long",
        color="cluster",
        mapbox_style="carto-positron"
)

fig.show(renderer="iframe")

In [38]:
filtered_result_75_cluster_2 = filtered_result_75_cluster

In [39]:
for i in np.arange(0.001,0.011,0.001):
    for j in np.arange(5,16,1):
    
        db = DBSCAN(eps=i, min_samples=j, metric="manhattan")

        db.fit(filtered_result_75_cluster_2)
    
        print('i=',i,'j=',j)
        print("nb de cluster:", len(np.unique(db.labels_)))
        print("nb -1:",len(filtered_result_75_cluster_2[filtered_result_75_cluster_2['cluster']==-1])/4225*100)
        
        filtered_result_75_cluster_2["cluster"] = db.labels_
        
        if len(np.unique(db.labels_))>1:
            print("metric:",metrics.silhouette_score(filtered_result_75_cluster_2,filtered_result_75_cluster_2['cluster']))
        

i= 0.001 j= 5
nb de cluster: 11
nb -1: 56.30769230769231


/var/folders/04/nhkbrdrd3453482rr7w_x_mc0000gn/T/ipykernel_29633/3878113447.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



metric: -0.7481365923007591
i= 0.001 j= 6
nb de cluster: 2
nb -1: 104.8284023668639


/var/folders/04/nhkbrdrd3453482rr7w_x_mc0000gn/T/ipykernel_29633/3878113447.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



metric: 0.2506577438665021
i= 0.001 j= 7
nb de cluster: 1
nb -1: 105.89349112426036
i= 0.001 j= 8
nb de cluster: 1
nb -1: 106.03550295857988
i= 0.001 j= 9
nb de cluster: 1
nb -1: 106.03550295857988


...

metric: 0.23322384862028614
i= 0.010000000000000002 j= 14
nb de cluster: 104
nb -1: 48.40236686390533


/var/folders/04/nhkbrdrd3453482rr7w_x_mc0000gn/T/ipykernel_29633/3878113447.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



metric: 0.13747848184791867
i= 0.010000000000000002 j= 15
nb de cluster: 89
nb -1: 54.10650887573964


/var/folders/04/nhkbrdrd3453482rr7w_x_mc0000gn/T/ipykernel_29633/3878113447.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



metric: 0.009980574387517845


In [40]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.01, min_samples=5, metric="manhattan")

db.fit(filtered_result_75)

DBSCAN(eps=0.01, metric='manhattan')

In [41]:
len(np.unique(db.labels_))

179

In [42]:
filtered_result_75_cluster = filtered_result_75
filtered_result_75_cluster["cluster"] = db.labels_

/var/folders/04/nhkbrdrd3453482rr7w_x_mc0000gn/T/ipykernel_29633/444248082.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
metrics.silhouette_score(filtered_result_75_cluster,filtered_result_75_cluster['cluster'])

0.8244212867391758

In [44]:
db.labels_.tolist().count(-1)

477

In [45]:
np.unique(db.labels_)

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177])

In [46]:
fig = px.scatter_mapbox(
        filtered_result_75_cluster[filtered_result_75_cluster.cluster != -1], 
        lat="lat", 
        lon="long",
        color="cluster",
        mapbox_style="carto-positron"
)

fig.show(renderer="iframe")